# AI Medicines Assitince

## 1. Imports and Initialization

In [22]:
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI as LangChainOpenAI
from langchain.agents.agent_types import AgentType
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
import tkinter as tk
from tkinter import scrolledtext, filedialog
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wavfile
import tempfile
import pyttsx3
import easyocr
import time
import traceback
from PIL import Image
from requests.adapters import HTTPAdapter, Retry
from gtts import gTTS
import io
import pygame
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from PyPDF2 import PdfReader

## 2. Load API Keys from .env

In [23]:
# Load environment variables
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 3. PDF Processing Functions

In [24]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

def preprocess_text(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    return splitter.split_text(text)

def setup_vector_db(texts):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store

def build_qa_chain_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if not text.strip():
        raise ValueError("Empty or unreadable PDF text.")
    chunks = preprocess_text(text)
    vector_store = setup_vector_db(chunks)
    retriever = vector_store.as_retriever()
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever), vector_store

##  4. OpenFDA API Integration

In [25]:
# OpenFDA Setup
session = requests.Session()
retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

def search_openfda(medicine_name):
    try:
        url = f"https://api.fda.gov/drug/label.json?search=openfda.brand_name:{medicine_name}&limit=1"
        response = session.get(url, timeout=5)
        data = response.json()

        if 'results' in data:
            result = data['results'][0]
            brand_name = result['openfda'].get('brand_name', ['Unknown'])[0]
            manufacturer = result['openfda'].get('manufacturer_name', ['Unknown'])[0]
            usage = result.get('indications_and_usage', ['No usage information'])[0]
            dosage = result.get('dosage_and_administration', ['No dosage information'])[0]
            warnings = result.get('warnings', ['No warnings'])[0]

            return f"""
Drug Information:
- Brand Name: {brand_name}
- Manufacturer: {manufacturer}
- Usage: {usage}
- Dosage: {dosage}
- Warnings: {warnings}
"""
        else:
            return "No information found in OpenFDA."
    except Exception as e:
        return f"Error fetching from OpenFDA: {e}"

## 5. LangChain Agent Setup

In [26]:
openfda_tool = Tool(
    name="search_openfda",
    func=search_openfda,
    description="Use this to get official medicine information like usage, dosage, warnings, or manufacturer. Always use for any medicine-related query."
)

tools = [openfda_tool]
llm = LangChainOpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

## 6. Text-to-Speech with gTTS

In [27]:
engine = pyttsx3.init()
def speak_text_gtts(text, lang='en'):
    try:
        tts = gTTS(text=text, lang=lang)
        fp = io.BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        pygame.mixer.init()
        pygame.mixer.music.load(fp, 'mp3')
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            continue
    except Exception as e:
        print(f"Voice playback error: {e}")

## 7. OCR (Image Text Extraction)

In [28]:
# EasyOCR
reader = easyocr.Reader(['en'], gpu=False)
def extract_drug_name(image_path):
    results = reader.readtext(image_path)
    if not results:
        return None
    texts_with_heights = [(text, abs(bbox[3][1] - bbox[0][1])) for bbox, text, conf in results]
    return max(texts_with_heights, key=lambda x: x[1])[0].lower().strip()

Using CPU. Note: This module is much faster with a GPU.


# 9. GPT-Powered Query Type Detection

In [29]:
# GPT Query Type Detection using OpenAI 
def detect_query_type_with_gpt(query):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a query classifier. Classify the user's query strictly as one of the following:\n"
                        "'medicine' – if it's asking about a drug name, dosage, effects, etc.\n"
                        "'symptom' – if it's asking about medical symptoms or conditions.\n"
                        "'other' – for anything unrelated.\n"
                        "Only respond with one word: 'medicine', 'symptom', or 'other'."
                    )
                },
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        print(f"Query type detection failed: {e}")
        return "other"


 ## 9. Audio Input with Whisper

In [30]:
# Audio Input (Whisper)
def recognize_speech_openai():
    fs = 16000
    seconds = 5
    print("Recording...")
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        wavfile.write(f.name, fs, (myrecording * 32767).astype(np.int16))
        with open(f.name, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return transcript.text

 ## 10. GUI App Using Tkinter

In [31]:
import tkinter as tk
from tkinter import scrolledtext, filedialog
from tkinter import font as tkfont
from tkinter import messagebox

class MedicineAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medicine Assistant")
        self.root.geometry("700x550")
        self.root.config(bg="#f0f0f0")  

        # Set a custom font for the window
        self.custom_font = tkfont.Font(family="Arial", size=12)

        # Frame for the conversation display
        frame = tk.Frame(self.root, bg="#f0f0f0")
        frame.pack(padx=20, pady=20, expand=True, fill="both")

        # Scrollable text widget for conversation display
        self.conversation_display = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=80, height=20, font=self.custom_font, bg="#f9f9f9", fg="#333")
        self.conversation_display.pack(pady=10)
        self.conversation_display.tag_configure("user", foreground="#1f77b4")  # User message color
        self.conversation_display.tag_configure("bot", foreground="#32cd32")   # Bot message color
        self.conversation_display.config(state=tk.DISABLED)

        # User input field
        self.user_input = tk.Entry(frame, width=70, font=self.custom_font, relief="solid", bd=2, highlightthickness=1, highlightbackground="#aaa")
        self.user_input.pack(pady=5)

        # Frame for buttons (Horizontal layout)
        button_frame = tk.Frame(frame, bg="#f0f0f0")
        button_frame.pack(pady=10)

        # Button for sending input
        self.send_button = tk.Button(button_frame, text="Send", font=("Arial", 14), command=self.handle_user_input, relief="solid", bd=2)
        self.send_button.grid(row=0, column=0, padx=10)

        # Button for speech input
        self.speech_button = tk.Button(button_frame, text="Speak", font=("Arial", 14), command=self.handle_speech_input, relief="solid", bd=2)
        self.speech_button.grid(row=0, column=1, padx=10)

        # Button for uploading an image
        self.upload_button = tk.Button(button_frame, text="Upload Image", font=("Arial", 14), command=self.handle_image_upload, relief="solid", bd=2)
        self.upload_button.grid(row=0, column=2, padx=10)

        # Button for uploading a PDF
        self.upload_pdf_button = tk.Button(button_frame, text="Upload PDF", font=("Arial", 14), command=self.handle_pdf_upload, relief="solid", bd=2)
        self.upload_pdf_button.grid(row=0, column=3, padx=10)

        self.qa_chain = None

    def display_message(self, message, is_user=False):
        self.conversation_display.config(state=tk.NORMAL)
        tag = "user" if is_user else "bot"
        prefix = "You: " if is_user else "Bot: "
        self.conversation_display.insert(tk.END, f"{prefix}{message}\n", tag)
        self.conversation_display.config(state=tk.DISABLED)
        self.conversation_display.yview(tk.END)

    def handle_user_input(self):
        user_query = self.user_input.get()
        if not user_query:
            messagebox.showwarning("Input Error", "Please enter a query!")
            return
        self.display_message(user_query, is_user=True)
        response = self.process_query(user_query)
        self.display_message(response)
        self.root.after(100, lambda: speak_text_gtts(response))  
        self.user_input.delete(0, tk.END)

    def process_query(self, query):
        query_type = detect_query_type_with_gpt(query)  
        if query_type == "symptom":
            return agent.run(query) 

        elif query_type == "medicine":
            try:
                return agent.run(query) 
            except Exception as e:
                return f"Agent error: {e}"

        elif self.qa_chain:
            try:
                return self.qa_chain.run(query)  
            except Exception as e:
                return f"PDF QA error: {e}"

        return "Sorry, I can only help with medicine or symptom-related questions."

    def handle_speech_input(self):
        try:
            text = recognize_speech_openai() 
            self.display_message(text, is_user=True)
            response = self.process_query(text)
            self.display_message(response)
            self.root.after(100, lambda: speak_text_gtts(response)) 
        except Exception as e:
            self.display_message(f"Speech error: {e}")

    def handle_image_upload(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp")])
        if not file_path:
            return
        self.display_message("Image uploaded. Extracting medicine name...")
        drug_name = extract_drug_name(file_path) 
        if drug_name:
            self.display_message(f"Detected Medicine Name: {drug_name}")
            try:
                result = agent.run(drug_name)  
                self.display_message(result)
                self.root.after(100, lambda: speak_text_gtts(result))  
            except Exception as e:
                self.display_message(f"Agent error: {e}")
        else:
            self.display_message("No recognizable drug name found.")

    def handle_pdf_upload(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if not file_path:
            return
        self.display_message("Uploading and processing PDF...")
        try:
            self.qa_chain, _ = build_qa_chain_from_pdf(file_path) 
            self.display_message("PDF loaded. You can now ask questions about its content.")
        except Exception as e:
            self.display_message(f"PDF error: {e}")

# Main
if __name__ == "__main__":
    root = tk.Tk()
    app = MedicineAssistantApp(root)
    root.mainloop()




> Entering new AgentExecutor chain...
 I should use search_openfda to get official medicine information
Action: search_openfda
Action Input: ibuprofeen
Observation: No information found in OpenFDA.
Thought: I now know the final answer
Final Answer: No information found in OpenFDA.

> Finished chain.


> Entering new AgentExecutor chain...
 I should use search_openfda to get official medicine information
Action: search_openfda
Action Input: ibuprofen
Observation: 
Drug Information:
- Brand Name: Ibuprofen Dye Free
- Manufacturer: CVS Pharmacy
- Usage: Uses temporarily relieves minor aches and pains due to: headache toothache backache menstrual cramps the common cold muscular aches minor pain of arthritis temporarily reduces fever
- Dosage: Directions do not take more than directed the smallest effective dose should be used adults and children 12 years and over: take 1 tablet every 4 to 6 hours while symptoms persist if pain or fever does not respond to 1 tablet, 2 tablets may be used 